# S&P 2019 Data Wrangling

**by Marc Angelo Acebedo**

## Table of Contents
- [Introduction](#introduction)
- [Data Wrangling](#wrangling)
    - [Gather](#gather)
    - [Assess](#assess)
    - [Clean](#clean)
    - [Store](#store)

## I) Introduction <a id = "introduction">

**Broad question:** How do price forecasts for each firm in the S&P 2019 Index compare to their corresponding actual prices?

**Approach:** Analyze difference in means between average forecast EPS and average actual EPS for each firm.

I will be analyzing quarterly price returns within the past 20 years for the firms present in the S&P 500 2019 Index.

> At first, I wanted to analyze the forecasted vs. actual price earnings of the S&P in its entirety for the past 20 years. However, considering that firms continuously enter and leave stock indices every year, there would be varying levels of inconsistencies and marginal errors when comparing annual S&P returns alone. To combat this problem, I have isolated these two approaches:
- Analyze the historical earnings of *only* the firms present in the S&P 2019 Index
- Keep track of all firms that were present in the S&P for the past 20 years. Keep track of how many times each firm appeared in the Index and for those with the least count, analyze them individually on how they differ from the firms that stayed for longer.

[TK] HEre is a breakdown of my final clean CSV's features, TK.csv.

## II) Data Wrangling <a id="wrangling"></a>

To gather the data depicted under the `./data` folder, I used Bloomberg Excel functions.

### A) Gather <a id = "gather"></a>
> **APPROACH 1:** Focus on the firms that appear in the 2019 S&P Index and analyze their forecasted vs. actual price earnings for the last 20 years.

To ensure consistency in analysis among multiple firms, I divide both the forecasted and actual price earning dates by *calendar period* instead of fiscal period. This is because fiscal period differs by firm whereas calendar period is consistent by dates. 

#### Through the Bloomberg Excel functions, I gathered four datasets with different purposes:

- historical forecasted EPS
- historical actual EPS
- historical actual EOD price
- historical forecasted EPS relying on terms

---
Before delving into the data, let's define what above terms:

**EPS**

> EPS stands for ***Estimated Price Earnings.*** The formal definition of EPS given by Investopedia is this:

Earnings per share is the portion of a company's profit that is allocated to each outstanding share of a common stock, serving as an indicator of the company's financial health.

In other words, the EPS is a portion of the company's **net income** after all of their dividends are paid off. Dividends are profits that are paid out to shareholders of the company. EPS is one of the most useful and valuable financial measurements because they ***determine a stock's worth.*** The higher the stock, the more the company can pay out dividends to its shareholders, and the more net profit they are determined to generate.

$$ EPS = \frac{Net Income - Preferred Dividends}{Weighted Average Common Shares Outstanding}\$$

**EOD**

> EOD stands for the ***End of Day*** price. For any given day, the EOD marks the ***price at which the stock was valued*** at the end of the day's trading period.

In [1]:
PATH = './data/'
PATH_CLEAN = './data/clean/'
PATH_CLEAN_AVGS = './data/clean/averages/'
PATH_CLEAN_AVGS_SUB = './data/clean/averages/components/'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from functools import reduce

### Import Data <a id = "import"></a>

### Let's summarize the contents of the following DataFrames as we import them.
> All DataFrames consist of the 505 firms found in the 2019 S&P Index with EPS and EOD data encompassing 20 years: from January 1999 until the December 2019.

**Historic forecasted EPS**
> According to Investopedia, consensus estimates is normally an average or median of all the forecasts from individual analysts tracking a particular stock. In this case, the consensus estimate is for ***EPS for each firm present in the index as of 2019.*** Forecasted EPS is calculated by ***quarterly earnings,*** usually by each firm's fiscal period. Estimates of quarterly earnings are published at the beginning of each quarterly period.

In [2]:
#historic forecasted EPS 
eps_fc = pd.read_csv(PATH + 'sp-eps-fc.csv')

In [3]:
eps_fc.head()

,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,1999Q1,NaN,NaN,NaN,0.025,NaN,NaN,-0.10,0.423,NaN,...,0.456,0.090,0.239,0.123,NaN,NaN,0.123,NaN,0.622,NaN
1,1999Q2,NaN,NaN,NaN,0.020,NaN,NaN,-0.13,0.420,NaN,...,0.213,0.090,0.264,0.135,NaN,NaN,0.150,NaN,0.661,NaN
2,1999Q3,NaN,NaN,NaN,0.023,NaN,NaN,-0.11,0.381,NaN,...,0.767,0.099,0.300,0.129,NaN,NaN,0.171,NaN,0.693,NaN
3,1999Q4,NaN,NaN,NaN,0.016,NaN,NaN,NaN,0.432,NaN,...,0.432,0.124,0.386,0.178,NaN,NaN,0.164,NaN,0.667,NaN
4,00Q1,NaN,NaN,NaN,0.032,NaN,NaN,NaN,0.441,NaN,...,0.283,0.143,0.436,0.138,NaN,NaN,0.138,NaN,0.668,NaN


**Historic actual EPS**
> Unlike forecasted EPS, actual EPS are the real numbers denoting Earnings-per-Share for a singular firm. Historic actual EPS will be compared to forecasted EPS to draw correlations and comparisons.

In [4]:
#historic actual EPS
eps_act = pd.read_csv(PATH + 'sp-eps-act.csv')

In [5]:
eps_act.head()

,Quarter,Year,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,Q1,1999,0.16,0.99,NaN,0.040000,NaN,0.0925,-0.030,0.44,...,0.34,0.0000,0.210,0.123333,1.64,NaN,0.1725,NaN,0.56,NaN
1,Q2,1999,0.35,1.76,NaN,0.035357,NaN,0.1000,-0.145,0.42,...,0.06,0.0975,0.285,0.133333,2.28,NaN,0.2900,NaN,0.60,NaN
2,Q3,1999,0.30,1.86,NaN,0.050357,NaN,0.1075,-0.135,0.30,...,0.63,0.0325,0.315,0.130000,1.96,NaN,0.3200,NaN,0.64,NaN
3,Q4,1999,0.32,1.89,NaN,0.024643,NaN,0.0375,-0.080,0.43,...,0.43,0.1300,0.600,0.180000,1.72,NaN,0.2375,NaN,0.49,NaN
4,Q1,2000,0.29,0.89,NaN,0.040585,NaN,0.1050,-0.045,0.45,...,0.45,0.1650,0.500,0.140000,-1.48,NaN,0.2025,NaN,-0.33,NaN


**Historic actual EOD**
> Though this is not directly related to EPS data, EOD would be an interesting measure to use when generating intriguing visualizations and analyses. Who knows what visuals and conclusions I would arrive to with this measure. 

In [6]:
#historic actual EOD
eod_act = pd.read_csv(PATH + 'sp-eod-act.csv')

In [7]:
eod_act.head()

,date,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,3/31/1999,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,NaN,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,6/30/1999,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,NaN,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,9/30/1999,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,NaN,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,12/31/1999,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,NaN,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,3/31/2000,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,NaN,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


**Historic forecasted EPS 3 months prior**
> Instead of using forecast data collected at the beginning of the fiscal period, this feature contains EPS data projected 3 months before the current fiscal period. This is an interesting metric to see how differently forecasters make their predictions at different times.

In [8]:
#historic forecasted EPS 3-months prior
eps_fc_terms = pd.read_csv(PATH + 'sp-eps-fc-terms.csv')

In [9]:
eps_fc_terms.head()

,Forecast Made,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,10/1/1999,00Q1,NaN,NaN,NaN,0.030,NaN,NaN,NaN,0.466,...,NaN,0.143,0.398,0.140,NaN,NaN,0.144,NaN,0.777,NaN
1,1/1/2000,00Q2,NaN,NaN,NaN,0.026,NaN,NaN,NaN,0.437,...,NaN,0.168,0.378,0.153,NaN,NaN,0.184,NaN,0.790,NaN
2,4/1/2000,00Q3,NaN,NaN,NaN,0.029,NaN,NaN,NaN,0.414,...,0.84,0.189,0.425,0.149,NaN,NaN,0.218,NaN,0.803,NaN
3,7/1/2000,00Q4,NaN,NaN,NaN,0.032,NaN,NaN,-0.18,0.477,...,0.53,0.211,0.487,0.200,NaN,NaN,0.244,NaN,0.777,NaN
4,9/1/2000,01Q1,NaN,NaN,NaN,0.041,NaN,NaN,-0.09,0.495,...,0.40,0.259,0.475,0.158,NaN,NaN,0.178,NaN,0.750,NaN


### B) Assess <a id = "assess"></a>

> The following DataFrames contain data for each firm across various dates. To account for all firm averages, my goal is to generate a CSV file where each row contains the firm average, with the features as columns.

In [10]:
dict_dfs = {'eps_fcast' : eps_fc,
           'eps_actual' : eps_act,
           'eod_actual' : eod_act,
           'eps_fcast_terms' : eps_fc_terms}

### Check Missing Data

In [11]:
for key, df in dict_dfs.items():
    print(key, df.shape)

eps_fcast (84, 506)
eps_actual (84, 507)
eod_actual (84, 506)
eps_fcast_terms (80, 507)


***We need to make sure the number of firms in each DataFrame is consistent.***

---

**Observation 1:** for `eps_fcast`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 506 columns: 1 column being `Term Forecast`, the rest firm names.

**Observation 2:** for `eps_actual`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 507 columns: 2 columns being `Quarter` and `Year`, the rest firm names.

**Observation 3:** for `eod_actual`, there are 505 firms encompassing 84 quarterly calendar periods since 1999.

> There are 506 columns: 1 column being `date`, the rest firm names.

**Observation 4:** for `eps_fcast_terms`, there are 505 firms encompassing 80 quarterly calendar periods since 1999.

> There are 507 columns: 2 columns being `Forecast Made` and `Term Forecasted`, the rest firm names.
 
Since there are only 80 quarterly calendar periods, that ***implies an entire year is missing.***

**Observation 5:** For `eps_fcast`, `eps_actual`, and `eod_actual`, since there are 4 quarters in a year, 84 quarterly forecast periods equate to 21 years. This is correct since we are analyzing the years from 1999 until the end of 2019.

### Most importantly, the number of firms across all DataFrames is consistent.

----

***Check which year is missing from `eps_fcast_terms`***.

> Each **Term Forecasted** entry under `eps_fcast_terms` records the year with 2 digits, so Quarter 1 of the year 2000 becomes 00Q1.
- isolate the first 2 characters to get the year
- join a '20' in front of the string so 00 becomes 2000
- list the number of unique values.

In [12]:
#iterate over all years in eps_fcast_terms, append '20' in front of string
list(map(lambda x: '20' + x, eps_fc_terms['Term Forecasted'].str[:2].unique()))

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

**Observation**: The year 1999 is missing from `eps_fc_terms`. This makes sense because the start of the forecasting period would be in the last quarter of 1999, which is October. 

---

### Check Nulls

In [13]:
for key, df in dict_dfs.items():
    #display (row, column) per DataFrame
    print('# NaN in {}: {}'.format(key, df.isna().sum().sum()))

# NaN in eps_fcast: 7055
# NaN in eps_actual: 5021
# NaN in eod_actual: 6921
# NaN in eps_fcast_terms: 7080


**Observation:** All four DataSets contain null values.
> In order to combat this, we'll have to look at both the **number of rows** and **number of columns** with missing data, separately. This way, we can isolate which firms and/or time periods contain complete or incomplete data.

In [14]:
#check rows for missing data
for key, df in dict_dfs.items():
    num_rows_missing = df.isna().sum().max()
    print('{} has {} time periods containing missing data, out of {} total rows.'.format(key, num_rows_missing, df.shape[0]))

eps_fcast has 82 time periods containing missing data, out of 84 total rows.
eps_actual has 84 time periods containing missing data, out of 84 total rows.
eod_actual has 83 time periods containing missing data, out of 84 total rows.
eps_fcast_terms has 80 time periods containing missing data, out of 80 total rows.


In [15]:
len(eps_fc.columns[eps_fc.isnull().any()])

254

In [16]:
#check columns for missing data
for key, df in dict_dfs.items():
    cols_missing = df.columns[df.isnull().any()]
    num_cols_missing = len(cols_missing)
    print('{} has {} firms containing missing data, out of {} total columns.'
         .format(key, num_cols_missing, df.shape[1]))

eps_fcast has 254 firms containing missing data, out of 506 total columns.
eps_actual has 432 firms containing missing data, out of 507 total columns.
eod_actual has 505 firms containing missing data, out of 506 total columns.
eps_fcast_terms has 340 firms containing missing data, out of 507 total columns.


**Observation 1:** The only two datasets with incomplete data for all time periods are **actual EPS** and **forecasted EPS 3 months prior.**

> To address this problem, it'd be helpful to isolate the time period ranges for the datasets with incomplete data by row, **forecasted EPS** and **actual EOD price.**

**Observation 2:** For all datasets, all firms contain incomplete data across all time periods.
> This is expected, as analyzing financial history spanning over 20 years will naturally be rife with missing and inaccurate data. The ***good news is that `eps_fcast`, `eps_actual`, and `eps_fcast_terms` are the most complete, while `eod_actual` contains the most amount of missing data.***


**Moving forward, we need to make sure that these inconsistencies won't clash with our analysis.**
> **My approach:** instead of looking at rows and columns ***with*** missing data, we'll be looking at rows and columns that ***are all missing data.***

I figured that if there is some missing data here and there scattered throughout the matrix, then that should not skew our analysis too much.

However, if there a significant amount of rows/columns that are entirely empty, then we ***might have to get ready to drop some dates and firms from our data overall.***

In [17]:
#check for empty rows, return False if row contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_rows = (df[cols_check].isnull().apply(lambda x: all(x), axis = 1)).value_counts()
    print(key, '\n', num_empty_rows, '\n----')

eps_fcast 
 False    84
dtype: int64 
----
eps_actual 
 False    84
dtype: int64 
----
eod_actual 
 False    84
dtype: int64 
----
eps_fcast_terms 
 False    80
dtype: int64 
----


**Observation:** All datasets do not contain empty rows.
> This is good news, since we can rely on the firms' averages per row instead of having to drop or limit time periods.

In [18]:
#check for empty columns, return False if column contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_cols = df[cols_check].isnull().apply(lambda x: all(x), axis = 0).value_counts()
    print(key, '\n', num_empty_cols)

eps_fcast 
 False    506
dtype: int64
eps_actual 
 False    506
True       1
dtype: int64
eod_actual 
 False    506
dtype: int64
eps_fcast_terms 
 False    506
True       1
dtype: int64


**Observation:** `eps_actual` and `eps_fcast_terms` are the only datasets that have an empty column.
> Let's isolate and look at the singular empty column for both DataFrames.

In [19]:
#create function to return an array of column names containing empty data
def comb_cols(df):
    empty_cols = []
    for column in df:
        if df[column].isnull().all():
            empty_cols.append(column)
            
    return empty_cols

In [20]:
#comb datasets for empty columns
print('In eps_act, the firm {} has no data.'.format(comb_cols(eps_act)))
print('In eps_fc_terms, the firm {} has no data.'.format(comb_cols(eps_fc_terms)))

In eps_act, the firm ['AMCR UN Equity'] has no data.
In eps_fc_terms, the firm ['AMCR UN Equity'] has no data.


**Observation:** The same firm in both datasets is empty of data.
> Though this is an annoying error to deal with, it still is to our advantage that both datasets ***share one firm*** in common for missing data. This way, we don't have to worry about dropping two entire firms.

---

### Check Duplicate Data

In [21]:
#check for duplicate data across all rows and columns
for key, df in dict_dfs.items():
    print(key, df.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate data.*** This is good news!

**Next, I will check for duplicated firm names.** Although the presence of duplicated firm names will inherently imply duplicated data, sometimes data gets dispersed in weird, unexpected ways, especially when dealing with large datasets.

In [22]:
#check for duplicated firm names
for key, df in dict_dfs.items():
    print(key, df.columns.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate firm names.*** This is also good news.

### There is no need to dedupe our data during the cleaning stage.

---

### Check Data Types

> Make sure all numerical data types are consistent.
- EPS and EOD values must all be **floats**
- Dates should all be **objects** (for now)

- `eps_fc` contains ***1 date field,***, so we should expect ***1 object type.***
- `eps_act` contains ***2 date fields,*** so we should expect ***2 object types.***
- `eod_act` contains ***1 date field,*** so we should expect ***1 object type.***
- `eps_fc_terms` contains ***2 date fields,*** so we should expect ***2 object types.***

> And everything else should be a ***float*** type.

**Examine each dataset's overall data types per column.**

In [23]:
for key, df in dict_dfs.items():
    print(key,'\n', df.dtypes.value_counts(), '\n--------')

eps_fcast 
 float64    505
object       1
dtype: int64 
--------
eps_actual 
 float64    499
object       7
int64        1
dtype: int64 
--------
eod_actual 
 float64    505
object       1
dtype: int64 
--------
eps_fcast_terms 
 float64    505
object       2
dtype: int64 
--------


**Observation 1:** `eps_fc`, `eod_act`, and `eps_fc_terms` are all consistent with the expected number of object types.

> But we'll still have to double check these facts later.

**Observation 2:** `eps_act` contains 5 more object columns than expected.

---

***Isolate 'Object' Columns under `eps_act`***

In [24]:
eps_act.select_dtypes(include = 'object').head()

,Quarter,BRK/B UN Equity,FOX UW Equity,GOOG UW Equity,HCP UN Equity,SYMC UW Equity,UA UN Equity
0,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
1,Q2,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
2,Q3,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
3,Q4,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
4,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable


**Observation:** These firms are all 'Object' types because there is no recorded data under them.
> This is actually missing data. From `eps_act`, there are 6 firms that are empty of data.

### This was a tricky situation to spot, but I caught it in the end. There actually is missing data after all, but we missed those 5 entire firms during the "Check Missing Data" stage, because they were recorded as objects. This means that Pandas wrongly recognized these 5 firms as "complete."

### Quality

**Missing Data**
> **In the end, I decided not to drop any rows or columns with missing data. For the sake of data preservation, I kept all missing data recorded as NaN.**

-  `eps_fc_terms` is missing the year 1999.
- `eps_act` and `eps_fc_act` have one firm with empty data: 'AMCR UN Equity'
- `eps_act` contains 7 empty firms: BRK/B UN Equity, FOX UW Equity, GOOG UW Equity, HCP UN Equity, SYMC UW Equity, UA UN Equity
--- 
- Unnormalized date formats among all DataFrames.
- Firm names across all DataFrames are capitalized and contain white space.
- Erroneous data type for `eps_act` Object columns.
- Erroneous data type for **date** under `eod_act`
- Erroneous data type for **forecast_made** under `eps_fc_terms`
- Erroneous data types for **term_forecast** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index
- No recorded quarterly data for `eod_act`
---
- Firm names not referenced by **firm_id**
- No recorded 20-year for each dataset.
- No recorded yearly averages for each dataset
- No recorded quarterly averages for each dataset.
---
- Features datasets 

### Tidiness
- Unnormalized data among `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs`
- 20-year, yearly, and quarterly averages contained in different DataFrames.
- Unnormalized data among `eps_act`, `eps_fc`, `eod_act`, and `eps_fc_terms`


### C) CLEAN <a id="clean"></a>

In [311]:
eps_fc_clean = eps_fc.copy()
eps_act_clean = eps_act.copy()
eod_act_clean = eod_act.copy()
eps_fc_terms_clean = eps_fc_terms.copy()

In [312]:
#create dictionary of all clean dfs for iteration
dict_clean = {'eps_fc_clean' : eps_fc_clean,
             'eps_act_clean' : eps_act_clean,
             'eod_act_clean' : eod_act_clean,
             'eps_fc_terms_clean' : eps_fc_terms_clean}

### Code
> **ISSUE 1:**  Unnormalized date formats among all DataFrames.

**Define:** 
- Conjoin `eps_act_clean` dates from 2 columns into 1 to match `eps_fc_clean` format.
> The format we want is YYYYQN. For example, Quarter 1 in 2005 will be 2005Q1.

- Rename both fiscal periods under `eps_act_clean` and `eps_fc_clean` to **term_forecast.**

- For both fiscal periods under `eps_fc_terms_clean`, add an underscore.
- Under `eps_fc_terms_clean`, format **term_forecasted** to YYYYQN.

In [313]:
#create new column, combine year and quarter into 1 string
eps_act_clean['term_forecast'] = eps_act_clean['Year'].map(str) + eps_act_clean['Quarter']

In [314]:
#drop 'Year' and 'Quarter'
eps_act_clean.drop(['Year', 'Quarter'], axis = 1, inplace = True)

In [315]:
#designate new column as first column
reorder_cols = eps_act_clean.columns.tolist()
reorder_cols.insert(0, reorder_cols.pop(reorder_cols.index('term_forecast')))

eps_act_clean = eps_act_clean.reindex(columns = reorder_cols)

In [316]:
#rename eps_fc_clean fiscal period to term_forecast
eps_fc_clean.rename(index = str, columns = {'Term Forecasted' : 'term_forecast'}, inplace = True)

#reassign to dictionary
dict_clean['eps_act_clean'] = eps_act_clean

In [317]:
eps_fc_clean[4:].term_forecast = '20' + eps_fc_clean[4:].term_forecast.astype(str)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [318]:
#add an underscore for both date periods under eps_fc_terms_clean
eps_fc_terms_clean.rename(index = str, columns = {'Forecast Made' : 'forecast_made',
                                                 'Term Forecasted' : 'term_forecast'},
                         inplace = True)

In [319]:
#under term_forecaste, add a '20' before each string
eps_fc_terms_clean.term_forecast = '20' + eps_fc_terms_clean.term_forecast.astype(str)

### Test

In [320]:
#give overview of both DataFrames
eps_fc_clean.sample(3)

,term_forecast,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
73,2017Q2,0.484,1.872,1.659,2.022,1.399,1.679,0.244,0.605,1.299,...,0.419,0.546,0.843,0.652,0.798,0.569,0.619,2.085,0.62,0.527
69,2016Q2,0.389,1.678,2.111,1.998,1.200,1.585,0.155,0.533,1.184,...,0.404,0.469,0.637,0.700,0.981,0.474,0.739,1.968,NaN,0.444
53,2012Q2,0.731,0.217,1.389,1.438,NaN,0.811,-0.091,1.217,0.860,...,0.355,0.437,1.953,0.557,NaN,0.489,0.704,1.317,NaN,NaN


In [321]:
eps_act_clean.sample(3)

,term_forecast,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
55,2012Q4,1.22,0.785075,0.890000,1.251429,0.976538,0.670,0.07,0.66,0.910000,...,0.29,0.46,2.20,0.89,1.04,0.392262,0.740000,0.88,0.19,-0.02
36,2008Q1,0.32,-1.370000,0.860000,0.258571,NaN,0.335,-0.26,0.61,0.597063,...,0.35,0.28,2.05,0.45,-1.08,NaN,0.522634,1.03,0.98,NaN
14,2002Q3,-0.49,-5.920000,0.266667,0.006429,NaN,0.215,-0.25,0.46,0.280000,...,-5.22,0.03,0.39,0.23,0.20,NaN,0.245000,0.33,0.43,NaN


In [322]:
#check consistent format between both `term_forecast` fields
forecasts = pd.concat([eps_fc_clean.term_forecast, eps_act_clean.term_forecast])

#count number of instances where unique term_forecast values are the same
(forecasts.value_counts().sort_index() == 2).value_counts()

True    84
Name: term_forecast, dtype: int64

In [323]:
#check that dates are the first columns
for key, df in dict_clean.items():
    print(df.columns[:2])

Index(['term_forecast', 'A UN Equity'], dtype='object')
Index(['term_forecast', 'A UN Equity'], dtype='object')
Index(['date', 'A UN Equity'], dtype='object')
Index(['forecast_made', 'term_forecast'], dtype='object')


## The **term_forecast** fields under `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` are all normalized to YYYQN format. This means `eod_act_clean` is the only one with a different date formatting rule.

### Code
> **ISSUE 2:** Firm names across all DataFrames are capitalized and contain white space.

**Define:** Iterate across all DataFrames. Get rid of everything from the first whitespace character onward. Lowercase column names.

In [324]:
#lowercase names
for key, df in dict_clean.items():
    lower_cols = [x.lower() for x in df.columns]
    df.columns = lower_cols

In [325]:
#get rid of white space
for key, df in dict_clean.items():
    stock_symbols = [x.split(' ', 1)[0] for x in df.columns]
    df.columns = stock_symbols

### Test

In [326]:
#check that all firm names are consistent, should return 505 firm results
all_ticks = []
for key, df in dict_clean.items():
    firm_ticks = np.array(df.columns.values)
    all_ticks.append(firm_ticks)
    
#flatten array
all_ticks = np.hstack(all_ticks)

#assign tick names to Pandas Series
all_ticks = pd.Series(all_ticks)

In [327]:
#look at lowest 3 value counts, SHOULD BE DATE PERIODS
all_ticks.value_counts().sort_values()[:5]

date             1
forecast_made    1
term_forecast    3
xec              4
mro              4
dtype: int64

In [328]:
#look at highest value counts, should be FOUR (because there are 4 dataframes all with firm names)
all_ticks.value_counts().sort_values(ascending = False)[:5]

jci     4
alxn    4
dte     4
cci     4
holx    4
dtype: int64

In [329]:
#check for any upper case instance among columns
all_ticks.str.isupper().any()

False

In [330]:
#check for any white space
all_ticks.str.contains(' ').any()

False

### Code
> **ISSUE 3:** Erroneous data type for `eps_act` Object columns.

**Define:**

- Convert '#N/A Field Not Applicable' strings into 'NaN'
- Convert the 6 `eps_act_clean` Object columns into NaN type.

In [331]:
#isolate 6 firm names recorded as 'object'
obj_ticks = eps_act_clean.select_dtypes(include = 'object').columns.values

#exclude 'term_forecast'
obj_ticks = obj_ticks[1:]

In [332]:
#convert to NaN
str_replace = '#N/A Field Not Applicable'
eps_act_clean[obj_ticks] = eps_act_clean[obj_ticks].replace(str_replace, np.nan)

### Test

In [333]:
eps_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, term_forecast to zts
dtypes: float64(505), object(1)
memory usage: 332.1+ KB


In [334]:
#make sure 'term_forecast' is the only object-type column
eps_act_clean.select_dtypes(include = 'object').columns.values.tolist()

['term_forecast']

### Code

> **ISSUE 4:**  Erroneous data type for **date** under `eod_act`

**Define:** Convert column `date` to DateTime object.

In [335]:
eod_act_clean.date = pd.to_datetime(eod_act_clean.date, infer_datetime_format=True)

### Test

In [336]:
eod_act_clean.sample(2)

,date,a,aal,aap,aapl,abbv,abc,abmd,abt,acn,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
39,2008-12-31,11.1768,NaN,33.65,12.1929,NaN,17.8300,16.420,25.5361,32.79,...,NaN,17.8200,79.83,28.2400,20.9977,NaN,22.6502,40.42,24.5100,NaN
5,2000-06-30,49.6906,NaN,NaN,3.7411,NaN,7.5201,15.375,19.9435,NaN,...,NaN,82.5625,39.25,10.2708,54.6678,NaN,5.0783,NaN,45.8906,NaN


In [337]:
#confirm datetime64 present
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, date to zts
dtypes: datetime64[ns](1), float64(505)
memory usage: 332.1 KB


### Code
> **ISSUE 5:** Erroneous data type for **forecast_made** under `eps_fc_terms`.

**Define:**

- Convert column **forecast_made** to DateTime object.

In [338]:
eps_fc_terms_clean.forecast_made = pd.to_datetime(eps_fc_terms_clean.forecast_made, infer_datetime_format=True)

### Test

In [339]:
#confirm presence of datetime64 object
eps_fc_terms_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 0 to 79
Columns: 507 entries, forecast_made to zts
dtypes: datetime64[ns](1), float64(505), object(1)
memory usage: 317.5+ KB


In [340]:
eps_fc_terms_clean.sample(2)

,forecast_made,term_forecast,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
21,2005-01-01,2005Q2,NaN,NaN,0.536,0.022,NaN,NaN,-0.03,0.600,...,0.203,0.240,0.88,0.330,NaN,NaN,0.306,0.705,1.210,NaN
12,2002-10-01,2003Q1,NaN,NaN,0.233,0.007,NaN,NaN,-0.34,0.585,...,0.280,0.116,0.52,0.225,NaN,NaN,0.217,0.333,0.965,NaN


### Code
> **ISSUE 6:** Erroneous data types for **term_forecast** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index

**Define:** Convert YYYYQQ formats into DateTime quarter index type

In [341]:
#helper function to turn a column into Quarter Period type
def convert_period(df, col):
    return pd.to_datetime(df[col]).dt.to_period('Q')
# eps_fc_terms_clean.term_forecast = pd.PeriodIndex(eps_fc_terms_clean.term_forecast, freq='Q')

In [342]:
#convert term_forecast fields to Period type
eps_fc_clean.term_forecast = convert_period(eps_fc_clean, 'term_forecast')
eps_act_clean.term_forecast = convert_period(eps_act_clean, 'term_forecast')
eps_fc_terms_clean.term_forecast = convert_period(eps_fc_terms_clean, 'term_forecast')

### Test

In [343]:
#verify Period Object types
print(eps_fc_clean.dtypes['term_forecast'])
print(eps_act_clean.dtypes['term_forecast'])
print(eps_fc_terms_clean.dtypes['term_forecast'])

period[Q-DEC]
period[Q-DEC]
period[Q-DEC]


### Code
> **ISSUE 7:** No recorded quarterly data for `eod_act`

**Define:** Add a new column **term_forecast** denoting *year* and *quarter*

In [344]:
eod_act_clean['term_forecast'] = eod_act_clean.date.dt.to_period('Q')

In [345]:
#reorder columns
eod_cols = ['date', 'term_forecast'] + [col for col in eod_act_clean.iloc[:, 1:] if col!='term_forecast']

In [346]:
eod_act_clean = eod_act_clean.reindex(columns = eod_cols)

### Test

In [347]:
#verify column order
eod_act_clean.head()

,date,term_forecast,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
0,1999-03-31,1999Q1,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,1999-06-30,1999Q2,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,1999-09-30,1999Q3,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,1999-12-31,1999Q4,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,2000-03-31,2000Q1,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


In [348]:
#verify 84 columns
eod_act_clean.shape[0]

84

In [349]:
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 507 entries, date to zts
dtypes: datetime64[ns](1), float64(505), period[Q-DEC](1)
memory usage: 332.8 KB


## All full years are normalized to YYYY-MM-DD, and all quarterly records to YYYY-QQ.

## This is to enable for more efficient handling, cleaning, and classifying of data later on.

### Code

**ISSUE 7:** Firm names not referenced by **firm_id**

**Define:**
- Assign a **firm_id** to each firm for future normalization.

- Generate new CSV named `firms.csv`

In [350]:
#get firm names
firm_names = eps_act_clean.columns[1:].to_numpy()

In [351]:
#assign ids to firm names
firm_names_ids = {}
counter = 0

for firm in firm_names:
    firm_names_ids.update({firm: counter})
    counter += 1

In [352]:
#generate new DF
firm_ids = pd.DataFrame(list(firm_names_ids.items())).rename(columns = {0: 'firm', 1: 'firm_id'})

In [353]:
#switch firm and firm_id columns
firm_ids = firm_ids.reindex(columns = ['firm_id', 'firm'])

### Test

In [354]:
firm_ids.sample(10)

,firm_id,firm
501,501,yum
140,140,dis
72,72,bll
427,427,swks
176,176,expe
4,4,abbv
301,301,ma
86,86,cbs
5,5,abc
453,453,uaa


In [355]:
firm_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 2 columns):
firm_id    505 non-null int64
firm       505 non-null object
dtypes: int64(1), object(1)
memory usage: 8.0+ KB


### Code
> **ISSUE 7:** No recorded 20-year averages for each dataset.

**Define:**


- Isolate 20-year averages for each firm into its own DataFrame

- Create new DataFrame `twenty_avgs` depicting all 20-year averages for each firm

In [356]:
#convert foreign keys to str temporarily
firm_ids_str = {str(k):str(v) for k, v in firm_names_ids.items()}
# firm_ids_str = {y:x for x, y in firm_ids_str.items()}

In [357]:
#helper function to get 20-year averages
def get_twenty_yr_avg(df, index_names, col_name, firm_ids_str):
    df = pd.DataFrame({col_name : df[index_names].mean()}).rename_axis('firm_id').reset_index()
    df['firm_id'] = df['firm_id'].map(firm_ids_str)
    return df

In [358]:
#get 20-year averages
eps_fc_twenty_avg = get_twenty_yr_avg(eps_fc_clean, firm_names, 'eps_fc', firm_ids_str)
eps_act_twenty_avg = get_twenty_yr_avg(eps_act_clean, firm_names, 'eps_act', firm_ids_str)
eod_act_twenty_avg = get_twenty_yr_avg(eod_act_clean, firm_names, 'eod_act', firm_ids_str)
eps_fc_terms_twenty_avg = get_twenty_yr_avg(eps_fc_terms_clean, firm_names, 'eps_fc_terms', firm_ids_str)

In [359]:
#put averages into a list
twenty_avg_list = [eps_fc_twenty_avg,
                  eps_act_twenty_avg,
                  eod_act_twenty_avg,
                  eps_fc_terms_twenty_avg]

In [360]:
#merge all Series
df_twenty_year_avgs = reduce(lambda x, y: pd.merge(x, y, on = 'firm_id', how = 'outer'), twenty_avg_list)

### Test

In [361]:
df_twenty_year_avgs.sample(5)

,firm_id,eps_fc,eps_act,eod_act,eps_fc_terms
461,461,1.011390,0.824578,79.508100,1.079532
296,296,0.336036,0.366125,22.804390,0.405065
405,405,0.407952,0.383253,32.041625,0.440250
364,364,0.690274,0.691766,40.314272,0.730132
141,141,0.373815,0.203342,22.501463,0.385870


In [362]:
#check there are 505 firms
df_twenty_year_avgs.firm_id.nunique()

505

In [363]:
#check missing data
df_twenty_year_avgs.isna().any()

firm_id         False
eps_fc          False
eps_act          True
eod_act         False
eps_fc_terms     True
dtype: bool

In [364]:
#check duplicate data
df_twenty_year_avgs.duplicated().any()

False

### Code
> **ISSUE 8:** No recorded yearly averages for each dataset.

**Define:**
- Create 4 separate DataFrames for all attributes
- Rename columns to "Feature_Year" (e.g. eps_fc_1999, eod_act_2000, etc.)
- Outer merge all DataFrames to create new DataFrame, `df_yearly_avgs`, on **firms** attribute

In [365]:
#helper function to calculate yearly average of each DataFrame
def calculate_df_yearly_avgs(df, col, firm_ids_str):
    #extract year from dates, reset index, and remove index name
    df = df.groupby(df[col].dt.year).mean().transpose().rename_axis('firm_id').reset_index().rename_axis(None, axis = 1)
    
    df['firm_id'] = df['firm_id'].map(firm_ids_str)
    
    return df

In [366]:
#helper function to melt columns
def melt_avgs(df, firm_id, var_name):
    return df.melt(id_vars = [firm_id],
                value_vars = df.columns.difference([firm_id]),
                var_name = var_name,
                value_name = 'average')

In [367]:
#calculate and melt
def create_df_yearly_avgs(df, col, firm_ids_str, firm_id):
    df = calculate_df_yearly_avgs(df, 'term_forecast', firm_ids_str)
    df = melt_avgs(df, firm_id, 'year')
    return df

In [368]:
#generate DataFrames of all yearly averages
eps_fc_yearly_avgs = create_df_yearly_avgs(eps_fc_clean, 'term_forecast', firm_ids_str, 'firm_id')
eps_act_yearly_avgs = create_df_yearly_avgs(eps_act_clean, 'term_forecast', firm_ids_str, 'firm_id')

#for eod_act, doesn't matter if we use date or term_forecast field
eod_act_yearly_avgs = create_df_yearly_avgs(eod_act_clean, 'date', firm_ids_str, 'firm_id')
eps_fc_terms_yearly_avgs = create_df_yearly_avgs(eps_fc_terms_clean, 'term_forecast', firm_ids_str, 'firm_id')

In [369]:
def store_df(path, df, csv_name):
    df.to_csv(path + csv_name, encoding = 'utf-8', index = False)

In [370]:
#store yearly averages
store_df(PATH_CLEAN_AVGS_SUB, eps_fc_yearly_avgs, 'yearly-avgs-eps-fc.csv')
store_df(PATH_CLEAN_AVGS_SUB, eps_act_yearly_avgs, 'yearly-avgs-eps-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, eod_act_yearly_avgs, 'yearly-avgs-eod-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, eps_fc_terms_yearly_avgs, 'yearly-avgs-eps-fc-terms.csv')

In [371]:
#add new column entailing "type" of feature before concat
eps_fc_yearly_avgs = eps_fc_yearly_avgs.assign(feature = 'eps_fc')
eps_act_yearly_avgs = eps_act_yearly_avgs.assign(feature = 'eps_act')
eod_act_yearly_avgs = eod_act_yearly_avgs.assign(feature = 'eod_act')
eps_fc_terms_yearly_avgs = eps_fc_terms_yearly_avgs.assign(feature = 'eps_fc_terms')

In [372]:
yearly_avgs_arr = [eps_fc_yearly_avgs, 
                  eps_act_yearly_avgs,
                  eod_act_yearly_avgs,
                  eps_fc_terms_yearly_avgs]

In [373]:
#merge all yearly average DataFrames
df_yearly_avgs = pd.concat(yearly_avgs_arr)

### Test

In [374]:
#each average, individually

#eps_fc_yearly
eps_fc_yearly_avgs.sample(10)

,firm_id,year,average,feature
6524,464,2011,1.34600,eps_fc
752,247,2000,NaN,eps_fc
5313,263,2009,0.30050,eps_fc
2720,195,2004,NaN,eps_fc
8983,398,2016,0.87000,eps_fc
6354,294,2011,0.34350,eps_fc
1960,445,2002,NaN,eps_fc
8312,232,2015,0.50600,eps_fc
4709,164,2008,0.91500,eps_fc
6851,286,2012,1.70525,eps_fc


In [375]:
#eps_fc_yearly
eps_act_yearly_avgs.sample(10)

,firm_id,year,average,feature
7778,203,2014,1.990000,eps_act
5351,301,2009,0.280038,eps_act
5385,335,2009,-0.093750,eps_act
7532,462,2013,0.987500,eps_act
1360,350,2001,0.090637,eps_act
9626,31,2018,0.787500,eps_act
9539,449,2017,-0.039320,eps_act
5979,424,2010,0.817500,eps_act
6134,74,2011,0.342500,eps_act
9194,104,2017,1.200000,eps_act


In [376]:
#eod_act_yearly
eod_act_yearly_avgs.sample(10)

,firm_id,year,average,feature
5769,214,2010,153.660000,eod_act
5940,385,2010,NaN,eod_act
6550,490,2011,NaN,eod_act
3257,227,2005,NaN,eod_act
1943,428,2002,NaN,eod_act
3788,253,2006,NaN,eod_act
7292,222,2013,76.360000,eod_act
5898,343,2010,NaN,eod_act
1915,400,2002,8.043750,eod_act
8076,501,2014,54.184325,eod_act


In [377]:
#eps_fc_terms_yearly
eps_fc_terms_yearly_avgs.sample(10)

,firm_id,year,average,feature
6071,11,2012,0.79825,eps_fc_terms
8833,248,2017,0.69775,eps_fc_terms
6338,278,2012,0.23825,eps_fc_terms
5655,100,2011,0.37450,eps_fc_terms
5111,61,2010,1.04700,eps_fc_terms
7454,384,2014,1.28550,eps_fc_terms
8547,467,2016,0.86400,eps_fc_terms
7402,332,2014,0.65900,eps_fc_terms
1578,63,2003,0.28900,eps_fc_terms
2788,263,2005,0.20425,eps_fc_terms


In [378]:
#verify all columns are consistent
print(eps_fc_yearly_avgs.columns, '\n',
      eps_act_yearly_avgs.columns, '\n',
      eod_act_yearly_avgs.columns, '\n',
      eps_fc_terms_yearly_avgs.columns)

Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'year', 'average', 'feature'], dtype='object')


In [379]:
#verify there are 505 firms in each dataframe
print(eps_fc_yearly_avgs.firm_id.nunique(), '\n',
      eps_act_yearly_avgs.firm_id.nunique(), '\n',
      eod_act_yearly_avgs.firm_id.nunique(), '\n',
      eps_fc_terms_yearly_avgs.firm_id.nunique())

505 
 505 
 505 
 505


In [380]:
#all yearly averages
df_yearly_avgs.sample(10)

,firm_id,year,average,feature
8482,402,2015,0.118333,eps_act
332,332,2000,NaN,eps_fc_terms
39,39,2000,NaN,eps_fc_terms
1248,238,2001,0.190000,eps_act
8871,286,2017,2.404750,eps_fc_terms
5582,27,2011,0.435250,eps_fc_terms
10235,135,2019,2.162750,eps_fc
1383,373,2001,15.582500,eod_act
3463,433,2005,0.508750,eps_fc
5063,13,2010,1.523250,eps_fc_terms


In [381]:
#verify 505 firms in all yearly averages df
df_yearly_avgs.firm_id.nunique()

505

### Code

> **ISSUE 9:** No recorded quarterly averages for each dataset.

**Define:** 

- Parse `eod_act_clean` dates  by **calendar quarter average** with quarterly data in a new DataFrame.
- Create separate DataFrames containing quarterly averages for `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` 
- Rename columns to "Quarter_Year_Feature" (e.g. **q1_eps_fc,** etc.)
- Outer merge all DataFrames into a new df `df_quarterly_avgs` on **firms**

In [382]:
#helper function to calculate quarterly average of each DataFrame
def calculate_df_quarterly_avgs(df, col, firm_ids_str):
    #extract quarter from Period objects
    df =  df.groupby(df[col].dt.strftime('Q%q')).mean().transpose().rename_axis('firm_id').reset_index().rename_axis(None, axis = 1)
    df['firm_id'] = df['firm_id'].map(firm_ids_str)
    
    return df

In [383]:
#calculate and melt
def create_df_quarterly_avgs(df, col, firm_ids_str, firm_id):
    df = calculate_df_quarterly_avgs(df, 'term_forecast', firm_ids_str)
    df = melt_avgs(df, firm_id, 'quarter')
    
    #lowercase quarter column
    df.quarter = df.quarter.str.lower()
    return df

In [384]:
#generate all quarterly average DataFrames
eps_fc_quarter_avgs = create_df_quarterly_avgs(eps_fc_clean, 'term_forecast', firm_ids_str, 'firm_id')
eps_act_quarter_avgs = create_df_quarterly_avgs(eps_act_clean, 'term_forecast', firm_ids_str, 'firm_id')
eod_act_quarter_avgs = create_df_quarterly_avgs(eod_act_clean, 'term_forecast', firm_ids_str, 'firm_id')
eps_fc_terms_quarter_avgs = create_df_quarterly_avgs(eps_fc_terms_clean, 'term_forecast', firm_ids_str, 'firm_id')

In [385]:
#store quarterly averages
store_df(PATH_CLEAN_AVGS_SUB, eps_fc_quarter_avgs, 'quarter-avgs-eps-fc.csv')
store_df(PATH_CLEAN_AVGS_SUB, eps_act_quarter_avgs, 'quarter-avgs-eps-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, eod_act_quarter_avgs, 'quarter-avgs-eod-act.csv')
store_df(PATH_CLEAN_AVGS_SUB, eps_fc_terms_quarter_avgs, 'quarter-avgs-eps-fc-terms.csv')

In [386]:
#add new column entailing "type" of feature before concat
eps_fc_quarter_avgs = eps_fc_quarter_avgs.assign(feature = 'eps_fc')
eps_act_quarter_avgs = eps_act_quarter_avgs.assign(feature = 'eps_act')
eod_act_quarter_avgs = eod_act_quarter_avgs.assign(feature = 'eod_act')
eps_fc_terms_quarter_avgs = eps_fc_terms_quarter_avgs.assign(feature = 'eps_fc_terms')

In [387]:
#put DataFrames in array
quarterly_avgs_arr = [eps_fc_quarter_avgs,
                  eps_act_quarter_avgs,
                  eod_act_quarter_avgs,
                  eps_fc_terms_quarter_avgs]

In [388]:
#merge all quarterly average DataFrames
df_quarter_avgs = pd.concat(quarterly_avgs_arr)

### Test

In [389]:
#each quarter individually

#eps_fc_quarter
eps_fc_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
1963,448,q4,0.188000,eps_fc
1675,160,q4,0.571143,eps_fc
846,341,q2,0.132667,eps_fc
1684,169,q4,-0.798095,eps_fc
411,411,q1,0.964810,eps_fc
1014,4,q3,1.361571,eps_fc
1864,349,q4,0.834762,eps_fc
80,80,q1,0.334476,eps_fc
444,444,q1,0.670952,eps_fc
1359,349,q3,0.825905,eps_fc


In [390]:
#eps_act_quarter
eps_act_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
1005,500,q2,0.481000,eps_act
1905,390,q4,0.426548,eps_act
1675,160,q4,-0.127061,eps_act
98,98,q1,-0.344944,eps_act
1577,62,q4,0.668000,eps_act
986,481,q2,0.406667,eps_act
1149,139,q3,0.643214,eps_act
1824,309,q4,0.416160,eps_act
497,497,q1,1.176667,eps_act
344,344,q1,1.768095,eps_act


In [391]:
#eod_act_quarter
eod_act_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
1370,360,q3,105.514857,eod_act
1830,315,q4,52.245565,eod_act
1322,312,q3,38.874545,eod_act
483,483,q1,46.210952,eod_act
926,421,q2,63.735000,eod_act
1087,77,q3,24.988662,eod_act
377,377,q1,72.662619,eod_act
1378,368,q3,57.865000,eod_act
1690,175,q4,37.172625,eod_act
59,59,q1,103.468238,eod_act


In [392]:
#eps_fc_terms_quarter
eps_fc_terms_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
1005,500,q2,0.574625,eps_fc_terms
1896,381,q4,0.510800,eps_fc_terms
1501,491,q3,0.401769,eps_fc_terms
1736,221,q4,0.305357,eps_fc_terms
10,10,q1,0.636929,eps_fc_terms
1163,153,q3,0.914750,eps_fc_terms
465,465,q1,0.585273,eps_fc_terms
585,80,q2,0.486200,eps_fc_terms
424,424,q1,0.717000,eps_fc_terms
1534,19,q4,2.274333,eps_fc_terms


In [393]:
#verify that all columns are consistent
print(eps_fc_quarter_avgs.columns, '\n',
      eps_act_quarter_avgs.columns, '\n',
      eod_act_quarter_avgs.columns, '\n',
      eps_fc_terms_quarter_avgs.columns)

Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object') 
 Index(['firm_id', 'quarter', 'average', 'feature'], dtype='object')


In [394]:
#verify there are 505 firms in each dataframe
print(eps_fc_quarter_avgs.firm_id.nunique(), '\n',
      eps_act_quarter_avgs.firm_id.nunique(), '\n',
      eod_act_quarter_avgs.firm_id.nunique(), '\n',
      eps_fc_terms_quarter_avgs.firm_id.nunique())

505 
 505 
 505 
 505


In [395]:
#all quarterly averages
df_quarter_avgs.sample(10)

,firm_id,quarter,average,feature
391,391,q1,0.018333,eps_act
236,236,q1,28.787095,eod_act
1429,419,q3,0.780227,eps_act
1880,365,q4,0.506700,eps_fc
96,96,q1,22.507162,eod_act
839,334,q2,33.074545,eod_act
1770,255,q4,23.448665,eod_act
957,452,q2,NaN,eps_act
343,343,q1,0.079934,eps_act
460,460,q1,0.802941,eps_fc_terms


In [396]:
#verify 505 firms in df_quarter_avgs
df_quarter_avgs.firm_id.nunique()

505

In [397]:
df_twenty_year_avgs.head(2)


,firm_id,eps_fc,eps_act,eod_act,eps_fc_terms
0,0,0.531678,0.352169,32.896718,0.567579
1,1,0.339879,-0.388520,40.303125,0.709745


### Code
>**ISSUE 10:** Unnormalized data among `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs`

**Define:** 
- Normalize features across all 3 DataFrames. 
- For `df_twenty_year_avgs`, melt **feature column names** (eps_fc, eps_act, eod_act, eps_fc_terms) into a single column called **feature**

Desired output for each DataFrame:


| firm_id     | average  | average_type | time_period   | feature|
|:------------|:-------------:|:-----------:|:---------:|-------:|
|43           | 3.14          | quarterly   | q1        | eps_fc |

> ***Apply the above table format to all three DFs, then concatenate to create `all_avgs` .***

**Note:** For `df_twenty_year_avgs`, it would be redundant to put *twenty_year* as the value under both **average_type** and **time_period** columns.

To curb this, I will assign all **time_period** values as NaN for `df_twenty_year_avgs`.

In [398]:
#assign column order
avgs_col_order = ['firm_id', 'average', 'average_type', 'time_period', 'feature']

In [399]:
#create clean copies of dfs
twenty_year_clean = df_twenty_year_avgs.copy()
yearly_clean = df_yearly_avgs.copy()
quarterly_clean = df_quarter_avgs.copy()

- ### `twenty_year_clean`

In [400]:
#isolate feature names
feats_col_names = twenty_year_clean.columns.difference(['firm_id'])

In [401]:
#move eps_fc . . . eps_fc_terms under "average_type"
twenty_year_clean = pd.melt(twenty_year_clean,
        id_vars = ['firm_id'],
        value_vars = feats_col_names,
        var_name = 'feature',
        value_name = 'average')

In [402]:
#add 'feature' and 'time_period' columns
twenty_year_clean = twenty_year_clean.assign(average_type = 'twenty_year', time_period = np.nan)

In [403]:
#reorder columns
twenty_year_clean = twenty_year_clean[avgs_col_order]

- ### `yearly_clean`

In [404]:
#assign 'average_type' column
yearly_clean = yearly_clean.assign(average_type = 'yearly')

In [405]:
#rename 'year' column to 'time_period'
yearly_clean.rename(columns = {'year' : 'time_period'}, inplace = True)

In [406]:
#reorder columns
yearly_clean = yearly_clean[avgs_col_order]

- ### `quarterly_clean`

In [407]:
#assign 'average_type' column
quarterly_clean = quarterly_clean.assign(average_type = 'quarterly')

In [408]:
#rename 'quarter' column to 'time_period'
quarterly_clean.rename(columns = {'quarter' : 'time_period'}, inplace = True)

In [409]:
#reorder columns
quarterly_clean = quarterly_clean[avgs_col_order]

### Code
>**ISSUE 11:** - 20-year, yearly, and quarterly averages contained in different DataFrames.

**Define:** Concatenate the normalized `twenty_year_clean`, `yearly_clean`, and `quarterly_clean` into new CSV **all_avgs.csv**.

In [410]:
#put all average DFs in an array
avgs_array = [twenty_year_clean, yearly_clean, quarterly_clean]

In [411]:
all_avgs = pd.concat(avgs_array)

### Test

In [412]:
all_avgs.sample(10)

,firm_id,average,average_type,time_period,feature
4188,148,34.757500,yearly,2007,eod_act
693,188,0.066250,yearly,2001,eps_fc_terms
1024,14,0.205550,quarterly,q3,eps_fc_terms
901,396,0.311397,twenty_year,NaN,eps_act
26,26,NaN,yearly,2000,eps_fc_terms
5409,359,0.393000,yearly,2010,eps_fc_terms
3208,178,9.787500,yearly,2005,eod_act
581,76,0.115000,yearly,2000,eps_act
8561,481,1.871000,yearly,2015,eps_fc
6959,394,NaN,yearly,2012,eod_act


In [413]:
#verify 
all_avgs.shape

(52015, 5)

In [414]:
#verify 505 firms, no duplicates
all_avgs.firm_id.nunique()

505

### Code
>**ISSUE 11:** Unnormalized data among `eps_act`, `eps_fc`, `eod_act`, and `eps_fc_terms`

**Define:**

- Achieve desired `features.csv` output:

| firm_id  | term_forecast     | value      | feature         |    date    |
|:---------| :-------: | :-----------------:| :-------------: |----------: |
| 44       |     2004Q1  |  3.55           |        eps_fc      |  NaN      |
| 55       | 2004Q4     |  6.22       |  eps_fc_terms       | 2004-07-01 |
| 202      | 2015Q1      | 6.28      |   eod_act         |   1999-03-31    |


> **date** feature contains the **date column in `eod_act`** and **forecast_made in `eps_fc_terms`**. They are assigned to the same column because both columns are consistent: the same DateTime Object.

> **date** feature will be NaN for `eps_fc` and `eps_act`.

- ### `eps_fc` and `eps_act`

> For both DataFrames, we don't have to worry about handling the **date** column. That will be resolved later when merging all the DataFrames with `eod_act` and `eps_fc_terms`

In [471]:
fc_act_order = ['firm_id', 'feature', 'term_forecast', 'value']

In [416]:
#melt firm names
eps_fc_clean = eps_fc_clean.melt (id_vars = ['term_forecast'],
                  var_name = 'firm_id')

In [417]:
eps_act_clean = eps_act_clean.melt(id_vars = ['term_forecast'],
                                  var_name = 'firm_id')

In [418]:
#reassign all firms to firm_id
eps_fc_clean['firm_id'] = eps_fc_clean['firm_id'].map(firm_names_ids)
eps_act_clean['firm_id'] = eps_act_clean['firm_id'].map(firm_names_ids)

In [419]:
#add feature column
eps_fc_clean = eps_fc_clean.assign(feature = 'eps_fc')
eps_act_clean = eps_act_clean.assign(feature = 'eps_act')

In [472]:
#reorder columns
eps_fc_clean = eps_fc_clean[fc_act_order]
eps_act_clean = eps_act_clean[fc_act_order]

- ### `eod_act`

In [473]:
#define column order
feats_col_order = ['firm_id', 'feature', 'date', 'term_forecast', 'value']

In [423]:
#melt firm ids
eod_act_clean = eod_act_clean.melt(id_vars = ['date', 'term_forecast'],
                  var_name = 'firm_id')

In [424]:
#reassign firm_ids
eod_act_clean['firm_id'] = eod_act_clean['firm_id'].map(firm_names_ids)

In [425]:
#add feature column
eod_act_clean = eod_act_clean.assign(feature = 'eod_act')

In [474]:
#reorder columns 
eod_act_clean = eod_act_clean[feats_col_order]

- ### `eps_fc_terms`

In [427]:
#melt firm ids
eps_fc_terms_clean = eps_fc_terms_clean.melt(id_vars = ['forecast_made', 'term_forecast'],
                       var_name = 'firm_id')

In [428]:
#reassign firm ids
eps_fc_terms_clean['firm_id'] = eps_fc_terms_clean['firm_id'].map(firm_names_ids)

In [429]:
#rename forecast_made to date
eps_fc_terms_clean = eps_fc_terms_clean.rename(columns = {'forecast_made' : 'date'})

In [430]:
#add feature column
eps_fc_terms_clean = eps_fc_terms_clean.assign(feature = 'eps_fc_terms')

In [475]:
#reorder columns
eps_fc_terms_clean = eps_fc_terms_clean[feats_col_order]

- ### `features.csv`

Now's the time to bring everything together!

In [432]:
feat_array = [eps_fc_clean, eps_act_clean, eod_act_clean, eps_fc_terms_clean]

In [476]:
#concat to prepare for outer left merge
eod_act_eps_terms = pd.concat([eod_act_clean, eps_fc_terms_clean], join = 'outer')

In [482]:
#outer join
features = pd.concat([eod_act_eps_terms, eps_fc_clean], join = 'outer', sort = False)
features = pd.concat([features, eps_act_clean], join = 'outer', sort = False)

In [483]:
features.sample(10)

,firm_id,feature,date,term_forecast,value
9210,109,eps_act,NaT,2012Q3,0.0175
15334,182,eod_act,2010-09-30,2010Q3,NaN
6967,82,eod_act,2018-12-31,2018Q4,127.0700
6546,77,eod_act,2018-09-28,2018Q3,42.7800
17989,214,eps_fc,NaT,2002Q2,1.0170
34112,406,eps_act,NaT,2001Q1,-0.3600
10178,121,eps_fc,NaT,2002Q3,0.0910
29698,353,eod_act,2010-09-30,2010Q3,NaN
14473,172,eps_fc,NaT,2005Q2,NaN
12069,143,eod_act,2013-06-28,2013Q2,42.5200


# III) Store Data <a id = "store"></a>

**Disclaimer:** I decided to put the following CSVs under the ***./data/clean/averages/components*** directory because they form *part* of the main CSVs under the ***./data/clean/averages*** directory:
- `quarter-avgs-eod-act.csv`
- `quarter-avgs-eps-act.csv`
- `quarter-avgs-eps-fc.csv`
- `quarter-avgs-eps-fc-terms.csv`

> The above are all components of the `quarter-avgs.csv` under **/averages/**

- `yearly-avgs-eod-act.csv`
- `yearly-avgs-eps-act.csv`
- `yearly-avgs-eps-fc.csv`
- `yearly-avgs-eps-fc-terms.csv`

> The above are all components of the `yearly-avgs.csv` under **/averages/**

**Additionally, I decided to keep `avgs.csv` under the *./data/clean/averages/* directory.** Although this CSV containing all averages would be highly inefficient for the data exploration stage, I decided it would be best to keep it on hand for convenience.

Using `avgs.csv` would mean having the parse through the column names to isolate 20-year, yearly, and quarterly averages, but all of that redundance can be avoided by simply focusing on the `twenty-year-avgs`, `yearly-avgs`, and `quarter-avgs` CSVs alone.

## Firms

In [199]:
firm_ids.to_csv(PATH_CLEAN + 'firms.csv', encoding = 'utf-8', index = False)

## Averages

In [200]:
#twenty-year-averages
df_twenty_year_avgs.to_csv(PATH_CLEAN_AVGS + 'twenty-year-avgs.csv', encoding = 'utf-8', index = False)

In [201]:
#yearly averagas
df_yearly_avgs.to_csv(PATH_CLEAN_AVGS + 'yearly-avgs.csv', encoding = 'utf-8', index = False)

In [202]:
#quarterly averages
df_quarter_avgs.to_csv(PATH_CLEAN_AVGS + 'quarter-avgs.csv', encoding = 'utf-8', index = False)

In [203]:
#all averages
all_avgs.to_csv(PATH_CLEAN_AVGS + 'avgs.csv', encoding = 'utf-8', index = False)

**NOTE:** For the twenty year, yearly, and quarterly datasets, I stored their old `df_` version instead of their new clean versions. 
> This is because their old versions are stored more efficiently as ***standalone*** CSVs, and were only modified to fit the larger `all_avgs` DataFrame.

## Features

In [484]:
eps_fc_clean.to_csv(PATH_CLEAN + 'eps-fc.csv')
eps_act_clean.to_csv(PATH_CLEAN + 'eps-act.csv')
eod_act_clean.to_csv(PATH_CLEAN + 'eod-act.csv')
eps_fc_terms_clean.to_csv(PATH_CLEAN + 'eps-fc-terms.csv')

features.to_csv(PATH_CLEAN + 'features.csv')

# IV) Notes



- When dealing with the CSVs depicting averages, we need to keep in mind **missing data** in our interpretations.
- I decided to put the years as columns under yearly average DF generation because: the amount of columns generated here, TK, would be less than the amount of 
	firms there are, 505. [TK link to a study showing how having too many columns take up memory vs rows]
    
- keep in mind that `eod_act` is based on **calendar years** not fiscal years.

In [136]:

#convert notebook to HTML